In [ ]:
!pip install Pillow

In [1]:
#Python Imaging Library
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
#do the standard imports
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

import numpy as np
import matplotlib.pyplot as plt

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10


def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  ## uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms.
    torch.backends.cudnn.enabled   = False  ## does not enable the inbuilt cudnn deep learning library for training neural networks

    return True

device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available(): ## run on GPUs if available
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")
#Use torchvision.datasets.CIFAR10 to load the CIFAR10 dataset
cifar10_train = CIFAR10("./", download=True, train=True)
cifar10_test = CIFAR10("./", download=True, train=False)


No GPU available!
Files already downloaded and verified
Files already downloaded and verified


In [2]:
#convert the dataset to a torch tensor
X_train = torch.tensor(cifar10_train.data).float()
#take only the first 50 images 
#and reshape to batchsize, channels, height, width
small_tensor = X_train[0:50,:].transpose(3,1).transpose(2,3)
single_image = small_tensor[0:1]
single_image.shape

torch.Size([1, 3, 32, 32])

In [3]:
#create the kernel
#out_channels, in_channels, kernel_height, kernel_width )
#out_channels: number of kernels, in_channels: number of channels in one kernel
SOBEL_VERTICAL = [[1, 0, -1],
                  [2, 0, -2],
                  [1, 0, -1]]
SOBEL_HORIZONTAL = [[1, 2, 1],
                    [0, 0, 0],
                    [-1, -2, -1]]
PREWITT_VERTICAL = [[1, 0, -1],
                    [1, 0, -1],
                    [1, 0, -1]]
single_image = small_tensor[0:1,:]
#single image with 4 axes
print(single_image.shape)
#create a kernel with 4 axes
kernel_3d = torch.tensor([SOBEL_VERTICAL, SOBEL_HORIZONTAL, PREWITT_VERTICAL]).float()
kernel_3d.unsqueeze_(0)
kernel_3d.shape

torch.Size([1, 3, 32, 32])


torch.Size([1, 3, 3, 3])

Devito set up

In [12]:
from abc import ABC, abstractmethod
from devito import Operator, Function
from numpy import array
from devito import Grid, Function, dimensions, Eq, Inc
import sympy
class Layer(ABC):
    def __init__(self, input_data):
        self._input_data = input_data
        self._R = self._allocate()

    @abstractmethod
    def _allocate(self) -> Function:
        # This method should return a Function object corresponding to
        # an output of the layer.
        pass

    def execute(self) -> (Operator, array):
        op = Operator(self.equations())
        op.cfunction

        return (op, self._R.data)

    @abstractmethod
    def equations(self) -> list:
        pass

2 Dimentional Convolution

In [13]:
class Subsampling(Layer):
    def __init__(self, kernel_size, feature_map, function,
                 stride=(1, 1), padding=(0, 0), activation=None,
                 bias=0):
        # All sizes are expressed as (rows, columns).

        self._error_check(kernel_size, feature_map, stride, padding)

        self._kernel_size = kernel_size
        self._function = function
        self._activation = activation
        self._bias = bias

        self._stride = stride
        self._padding = padding

        super().__init__(input_data=feature_map)

    def _error_check(self, kernel_size, feature_map, stride, padding):
        if feature_map is None or len(feature_map) == 0:
            raise Exception("Feature map must not be empty")

        if kernel_size is None or len(kernel_size) != 2:
            raise Exception("Kernel size is incorrect")

        if stride is None or len(stride) != 2:
            raise Exception("Stride is incorrect")

        if stride[0] < 1 or stride[1] < 1:
            raise Exception("Stride cannot be less than 1")

        if padding is None or len(padding) != 2:
            raise Exception("Padding is incorrect")

        if padding[0] < 0 or padding[1] < 0:
            raise Exception("Padding cannot be negative")

        map_height = len(feature_map) + 2 * padding[0]
        map_width = len(feature_map[0]) + 2 * padding[1]
        kernel_height, kernel_width = kernel_size

        if (map_height - kernel_height) % stride[0] != 0 or \
           (map_width - kernel_width) % stride[1] != 0:
            raise Exception("Stride " + str(stride) + " is not "
                            "compatible with feature map, kernel and padding "
                            "sizes")

    def _allocate(self):
        map_height = len(self._input_data) + 2 * self._padding[0]
        map_width = len(self._input_data[0]) + 2 * self._padding[1]
        kernel_height, kernel_width = self._kernel_size

        gridB = Grid(shape=(map_height, map_width))
        B = Function(name='B', grid=gridB, space_order=0)

        a, b = dimensions('a b')
        gridR = Grid(shape=((map_height - kernel_height + self._stride[0])
                            // self._stride[0],
                            (map_width - kernel_width + self._stride[1])
                            // self._stride[1]),
                     dimensions=(a, b))
        print(gridR)
        R = Function(name='R', grid=gridR, space_order=0)

        for i in range(self._padding[0], map_height - self._padding[0]):
            B.data[i] = \
                np.concatenate(([0] * self._padding[1],
                                self._input_data[i - self._padding[0]],
                                [0] * self._padding[1]))

        self._B = B
        print("b shape", B.shape)
        return R

    def equations(self):
        a, b = self._B.dimensions
        kernel_height, kernel_width = self._kernel_size

        rhs = self._function([self._B[self._stride[0] * a + i,
                                      self._stride[1] * b + j]
                              for i in range(kernel_height)
                              for j in range(kernel_width)]) + self._bias

        if self._activation is not None:
            rhs = self._activation(rhs)

        return [Eq(self._R[a, b], rhs)]
                
Sample_obj = Subsampling((2,2),single_image[0][0], lambda l: sympy.Max(*l))
#A.equations()
tupC = Sample_obj.execute()
tupC[0].apply()
tupC[1].shape

Grid[extent=(1.0, 1.0), shape=(31, 31), dimensions=(a, b)]
b shape (32, 32)


Operator `Kernel` run in 0.01 s


(31, 31)

Convolution in 4 axes

In [14]:
class Subsampling_4d(Layer):
    def __init__(self, kernel_size, feature_map, function,
                 stride=(1, 1), padding=(0, 0), activation=None,
                 bias=0):
        # All sizes are expressed as (rows, columns).

        #self._error_check(kernel_size, feature_map, stride, padding)

        self._kernel_size = kernel_size
        self._function = function
        self._activation = activation
        self._bias = bias

        self._stride = stride
        self._padding = padding

        super().__init__(input_data=feature_map)


    def _allocate(self):
        map_height = self._input_data.shape[2] + 2 * self._padding[0]
        map_width = self._input_data.shape[3] + 2 * self._padding[1]
        kernel_height, kernel_width = self._kernel_size
        a, b, c, d = dimensions('a b c d')
        gridB = Grid(shape=(self._input_data.shape[0], self._input_data.shape[1], map_height, map_width),\
                    dimensions=(a, b, c, d))
        B = Function(name='B', grid=gridB, space_order=0)

        e, f, g, h = dimensions('e f g h')
        gridR = Grid(shape=( self._input_data.shape[0],  self._input_data.shape[1],\
                            (map_height - kernel_height + self._stride[0])
                            // self._stride[0],
                            (map_width - kernel_width + self._stride[1])
                            // self._stride[1]),
                     dimensions=(e, f, g, h))
        print(gridR)
        R = Function(name='R', grid=gridR, space_order=0)
        #add padding to start and end of each row
        for image in range(self._input_data.shape[0]):
            for channel in range(self._input_data.shape[1]):
                for i in range(self._padding[0], map_height - self._padding[0]):
                    B.data[image, channel, i] = \
                        np.concatenate(([0] * self._padding[1],
                                        self._input_data[image, channel, i - self._padding[0]],
                                        [0] * self._padding[1]))

        self._B = B
        print("b dim", self._B.dimensions)
        print("b self shape", self._B.shape)
        print("b  shape", B.shape)
        return R

    def equations(self):
        a, b, c, d = self._B.dimensions
        kernel_height, kernel_width = self._kernel_size
        images = self._input_data.shape[0]
        channels = self._input_data.shape[1]
        print(channels)
        rhs = self._function([self._B[image, channel, self._stride[0] * c + i,
                                      self._stride[1] * d + j]
                              for image in range(images)
                              for channel in range(channels)
                              for i in range(kernel_height)
                              for j in range(kernel_width)
                              ]) + self._bias

        if self._activation is not None:
            rhs = self._activation(rhs)

        return [Eq(self._R[a, b, c, d], rhs)]

Sample_obj4 = Subsampling_4d((2,2),single_image, lambda l: sympy.Max(*l))
#A.equations()
tup4 = Sample_obj4.execute()
tup4[0].apply()
tup4[1].shape

/app/devito/types/grid.py:206: RuntimeWarning: divide by zero encountered in true_divide
  spacing = (np.array(self.extent) / (np.array(self.shape) - 1)).astype(self.dtype)


Grid[extent=(1.0, 1.0, 1.0, 1.0), shape=(1, 3, 31, 31), dimensions=(e, f, g, h)]
b dim (a, b, c, d)
b self shape (1, 3, 32, 32)
b  shape (1, 3, 32, 32)
3


Operator `Kernel` run in 0.01 s


(1, 3, 31, 31)

In [18]:
# each channel has the same values
np.allclose(tup4[1][0][0],tup4[1][0][1])

True